# **Face Recognition on the webcam**

In [1]:
import os
import cv2
from PIL import Image
import numpy as np

In [70]:
def convert_to_grayscale(img_path, save_dir=None):
    gray_img = None  
    valid_exts = ('.jpg', '.jpeg', '.png', '.bmp')

    if os.path.isfile(img_path) and img_path.lower().endswith(valid_exts):
        # Load the image with PIL
        img = Image.open(img_path)

        # Convert to grayscale
        gray_img = img.convert("L")

        # Convert PIL image to NumPy array for OpenCV display
        gray_np = np.array(gray_img)

        # Show the image using OpenCV
        cv2.imshow("Grayscale Image", gray_np)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # Save the image if save_dir is provided
        if save_dir:
            os.makedirs(save_dir, exist_ok=True)
            filename = os.path.basename(img_path)
            save_path = os.path.join(save_dir, filename)
            gray_img.save(save_path)

    return gray_img

In [68]:
source_dir = 'C:/Users/Admin/Documents/CVs/img/me/face/'
target_dir = 'C:/Users/Admin/Documents/CVs/img/me/face/gray'

In [76]:
paths = [os.path.join(source_dir, f) 
         for f in os.listdir(source_dir) 
         if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

for path in paths:
    convert_to_grayscale(path, target_dir)

In [ ]:
# train the model to get the face
folder_path = 'C:/Users/Admin/Documents/CVs/img/me/face/gray'

def get_img_data():
  valid_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
  paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path)]
  ids = []
  faces = []
  for path in paths:
    if os.path.isfile(path) and path.lower().endswith(valid_exts):
      img = Image.open(path).convert('L') # L is single scale color image
      img_np = np.array(img, 'uint8')
      # Assign a fixed ID (e.g., 1) to all images
      id = 1
      ids.append(id)
      faces.append(img_np)

  return np.array(ids), faces

In [ ]:
# get the faces and id of the faces
ids, faces = get_img_data()

# create classifier instance and input training model to output
lbph_classifier = cv2.face.LBPHFaceRecognizer_create()
lbph_classifier.train(faces, ids)
lbph_classifier.write('lbph_classifier.yml')


In [17]:
# khởi tạo Haar Cascase Classifier nhận diện khuôn mặt
face_detector = cv2.CascadeClassifier('./data/haarcascade_frontalface_default.xml')
# # khởi tạo HCC nhận diện mắt
# eye_cascade = cv2.CascadeClassifier('data/haarcascade_eye_tree_eyeglasses.xml')
# # khởi tạo HCC nhận diện miệng
# smile_cascade = cv2.CascadeClassifier('data/haarcascade_smile.xml')

face_recognizer= cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read('data/lbph_classifier.yml')

In [19]:
width, height = 220, 200
font =cv2.FONT_HERSHEY_COMPLEX_SMALL
camera = cv2.VideoCapture(0)

while (True):
    con, image = camera.read()
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    detections = face_detector.detectMultiScale(img_gray, scaleFactor=1.2, minNeighbors=3, minSize=(30, 30))
    for (x,y,w,h) in detections:
        img_face = cv2.resize(img_gray[y:y + w, x:x + h], (width, height))
        cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
        id, confi = face_recognizer.predict(img_face)
        name = ""
        if id == 1:
            name = 'Duong'
        elif id == 2:
            name = 'Phuc'
        cv2.putText(image, name,  (x,y + (w+30)), font, 2, (0,0,255))
        cv2.putText(image, str(confi),  (x,y + (h+50)), font, 1, (255,0,255))

    cv2.imshow("Face", image)
    if cv2.waitKey(1) == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()